In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import sys
sys.path.append("..")

In [2]:
import numpy as np
from utils.helper import get_encoding_data
from utils.evaluate import evaluation_metrics

In [26]:
X, y, X_test = get_encoding_data(fmri_dir="../data/participants_data_v2021",
                                 activations_dir='../data/features/alexnet/pca_250',
                                 layer_id="layer_1",
                                 subject_id="sub01",
                                 roi_type="V1")
X_train, X_val = X[:900], X[900:]
y_train, y_val = y[:900], y[900:]

FileNotFoundError: [Errno 2] No such file or directory: '../data/features/alexnet/pca_250/train_layer_1.npy'

In [23]:
import glob
layer = "layer_5"
activations_dir = "../data/features/alexnet/activations"
activations_file_list = glob.glob(activations_dir +'/*' + layer + '.npy')
activations_file_list.sort()
feature_dim = np.load(activations_file_list[0]).shape[0]
x = np.zeros((len(activations_file_list), feature_dim))
for i, activation_file in enumerate(activations_file_list):
    temp = np.load(activation_file)
    x[i,:] = temp
X = x[:1000,:]
X_train, X_val = X[:900], X[900:]
X_train.shape

# Partial Least Squares

In [25]:
from encoding_models.partial_ls import fit_partial_ls, predict_partial_ls
model_config = {"n_components": 25}
model_params = fit_partial_ls(model_config, X_train, y_train)
y_pred = predict_partial_ls(model_params, X_val)
# corr, mse, mae
evaluation_metrics(y_val, y_pred)

(0.17433227896313327, 0.26111848934290827, 0.40482662178650825)

# Gradient Boosted Trees - Sklearn Version

In [31]:
from encoding_models.trees import fit_gradboost_model, predict_gradboost_model
model_config = {"alpha": 0.05, "learning_rate": 0.05, "max_depth": 10, "n_estimators": 50}
model_params = fit_gradboost_model(model_config, X_train, y_train)
y_pred = predict_gradboost_model(model_params, X_val)
# corr, mse, mae
evaluation_metrics(y_val, y_pred)

(0.08204222318962938, 0.26505313191777513, 0.40837804329311156)

# Elastic Net - Penalized Regression

In [17]:
from encoding_models.elastic_net import fit_elastic_net, predict_elastic_net
model_config = {"alpha": 0.025, "l1_ratio": 0.5}
model_params = fit_elastic_net(model_config, X_train, y_train)
y_pred = predict_elastic_net(model_params, X_val)
# corr, mse, mae
evaluation_metrics(y_val, y_pred)

(0.1370565067476729, 0.25007557346167286, 0.3961864815989181)

# Linear Regression

In [32]:
from encoding_models.ols import fit_linear_model, predict_linear_model
model_config = {"lambda_reg": 0.0}
model_params = fit_linear_model(model_config, X_train, y_train)
y_pred = predict_linear_model(model_params, X_val)
# corr, mse, mae
evaluation_metrics(y_val, y_pred)

(0.07998744740767788, DeviceArray(0.32005557, dtype=float32), 0.43471348)

# Multi-Output MLP Networks

In [34]:
from encoding_models.mlp_networks import fit_mlp_model, predict_mlp_model
model_config = {"num_hidden_layers": 2,
                "num_hidden_units": 64,
                "hidden_act": "relu",
                "optimizer_class": "Adam",
                "learning_rate": 3e-04,
                "w_decay": 1e-07,
                "dropout": 0.0}
model_params = fit_mlp_model(model_config, X_train, y_train)
y_pred = predict_mlp_model(model_params, X_val)
# corr, mse, mae
evaluation_metrics(y_val, y_pred)

(0.0898770722254375, 0.2556076024462942, 0.4001329879328703)